# Central Bank Policy Rates - BIS

## Python set-up

In [1]:
# system imports
from pathlib import Path
from urllib.error import HTTPError, URLError
from typing import Any
import textwrap as tw
from collections.abc import Iterable

In [2]:
# analytic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycountry

In [3]:
# local imports
from mgplot import (
    set_chart_dir,
    clear_chart_dir,
    line_plot_finalise,
    finalise_plot,
)

In [4]:
# plotting set-up
SOURCE = "Source: BIS policy rates"
LFOOTER = "Daily data.  Note: There are lags in BIS data reporting. "
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/BIS/"
set_chart_dir(CHART_DIR)
clear_chart_dir()
SHOW = False

## Data capture

In [5]:
def country_code_to_name(country_code: str) -> str:
    """Convert 2-digit country codes to country names."""

    try:
        country = pycountry.countries.get(alpha_2=country_code)
        return country.name
    except AttributeError:
        if country_code == "XM":
            return "Euro Area"
        return country_code

In [6]:
def index_missing_dates(data: pd.DataFrame | pd.Series) -> pd.DataFrame | pd.Series:
    """Reindex [and sort] a Series/DataFrame to include all missing dates in the index.
    This function works for data with either a DatetimeIndex or PeriodIndex."""

    # check that the index is a DatetimeIndex or PeriodIndex
    assert isinstance(data.index, pd.DatetimeIndex) or isinstance(
        data.index, pd.PeriodIndex
    )

    function = (
        pd.period_range if isinstance(data.index, pd.PeriodIndex) else pd.date_range
    )
    index = function(start=data.index.min(), end=data.index.max(), freq="D")
    data = data.reindex(index, fill_value=np.nan)
    data = data.sort_index()
    data = data.ffill()

    return data

In [7]:
# takes a few minutes to run
import pickle
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def get_bis_data(start="2018-01-01", end=None) -> tuple[pd.DataFrame, str]:
    """
    Get central bank policy rates from the BIS for a select set of states.
    Arguments: 
        start -- the start date for the data (default: 2018-01-01)
        end -- the end date for the data (default: None, which means latest)
    Returns a DataFrame of daily data and a string of the latest rates.
    """

    states = sorted(
        list(
            set(
                [  # ensure unique and sorted
                    "AU",
                    "CA",
                    "GB",
                    "JP",
                    "NO",
                    "KR",
                    "NZ",
                    "SE",
                    "US",
                    "XM",
                    "AR",
                    "BR",
                    "CL",
                    "CN",
                    "CZ",
                    "DK",
                    "HK",
                    "HU",
                    "IN",
                    "ID",
                    "IL",
                    "MY",
                    "MX",
                    "PH",
                    "PL",
                    "RU",
                    "ZA",
                    "TH",
                    "TR",
                    "CO",
                    "HR",
                    "IS",
                    "MA",
                    "MK",
                    "PE",
                    "RO",
                    "SA",
                    "RS",
                    "CH",
                ]
            )
        )
    )
    box = {}
    finals = []
    for abbr in states:
        for _trys in range(2):
            state = country_code_to_name(abbr)
            url = f"https://stats.bis.org/api/v2/data/dataflow/BIS/WS_CBPOL/1.0/D.{abbr}?startPeriod={start}"
            if end:
                url += f"&endPeriod={end}"
            url += "&format=csv"
            try:
                df = pd.read_csv(url)[["TIME_PERIOD", "OBS_VALUE"]]
            except (HTTPError, URLError) as e:
                print(f"Internet Error: {state} {e}")
                continue
            except Exception as e:
                # Empty dataset returns error
                continue
            if df.empty:
                continue
            s = pd.Series(
                df["OBS_VALUE"].values,
                name=state,
                dtype=float,
                index=pd.to_datetime(df["TIME_PERIOD"]),
            )
            if s.empty or s.isnull().all():
                print(f"Empty: {state}")
                continue
            s = index_missing_dates(s)  # type: ignore[assignment]
            box[state] = s
            finals.append(f"{abbr}: {s.iloc[-1]:.2f}")
            break

    if not box:
        return pd.DataFrame(), ""
    
    data = pd.DataFrame(box)
    data.index = pd.PeriodIndex(data.index, freq="D")

    return data, ", ".join(finals)

# Smart caching: update existing cache with recent data
cache_file = Path("./bis_data_cache.pkl")
cache_age_limit = timedelta(hours=24)  # Check for updates every 24 hours

try:
    if cache_file.exists():
        cache_time = datetime.fromtimestamp(cache_file.stat().st_mtime)
        print(f"Cache found (age: {datetime.now() - cache_time})")
        
        # Load existing cache
        with open(cache_file, 'rb') as f:
            df_cached, _ = pickle.load(f)
        
        if datetime.now() - cache_time < cache_age_limit:
            print("Using cached data (checked recently)")
            df = df_cached
        else:
            print("Updating cache with recent data...")
            # Get the last date in cache
            last_cached_date = pd.to_datetime(df_cached.index[-1].strftime('%Y-%m-%d'))
            
            # Fetch data from 3 months before last cached date to ensure we capture any revisions
            update_from = (last_cached_date - relativedelta(months=3)).strftime('%Y-%m-%d')
            print(f"Fetching data from {update_from} onwards (3 months overlap)...")
            df_update, _ = get_bis_data(start=update_from)
            
            if not df_update.empty:
                # Combine cached and new data (new data takes precedence for overlapping dates)
                df = df_cached.combine_first(df_update)
                # Ensure all columns from both dataframes are present
                for col in df_update.columns:
                    if col not in df.columns:
                        df[col] = df_update[col]
                # Update overlapping data with fresh values (in case of revisions)
                for col in df_cached.columns:
                    if col in df_update.columns:
                        df.loc[df_update.index, col] = df_update[col]
                df = df.sort_index()
                print("Cache updated with recent data")
            else:
                print("No new data available, using existing cache")
                df = df_cached
            
            # Save updated cache
            latest_rates = ", ".join([f"{col[:2]}: {df[col].iloc[-1]:.2f}" for col in df.columns if not pd.isna(df[col].iloc[-1])])
            with open(cache_file, 'wb') as f:
                pickle.dump((df, latest_rates), f)
    else:
        print("No cache found, fetching full dataset from BIS...")
        df, latest_rates = get_bis_data()  # takes around 3 minutes for full dataset
        if not df.empty:
            with open(cache_file, 'wb') as f:
                pickle.dump((df, latest_rates), f)
            print("Full dataset fetched and cached")
        else:
            raise ValueError("Failed to fetch any data from BIS")
            
    # Generate latest_rates string from the data
    latest_rates = ", ".join([f"{col[:2]}: {df[col].iloc[-1]:.2f}" for col in df.columns if not pd.isna(df[col].iloc[-1])])
    
except Exception as e:
    print(f"Error with data fetching: {e}")
    # Try to use stale cache if available
    if cache_file.exists():
        print("Using stale cache due to error...")
        with open(cache_file, 'rb') as f:
            df, latest_rates = pickle.load(f)
    else:
        raise

print(f"Data shape: {df.shape}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
print(df.index.dtype)

Cache found (age: 0:07:08.992479)
Using cached data (checked recently)
Data shape: (2795, 39)
Date range: 2018-01-01 to 2025-08-26
period[D]


### By hand adjustments

In [8]:
# Make adjustments when BIS data is lagged


adjustments = {
    # Nation : [adjustment, date-as-period]
    "Australia": [3.6, pd.Period("2025-08-13", freq="D")],
    "New Zealand": [3.0, pd.Period("2025-08-20", freq="D")],
}


def make_adjustments(frame: pd.DataFrame, adjust_dict: dict) -> pd.DataFrame:
    """Because the BIS data is highly lagged, we may need to adjust it for late
    policy rate changes.
    Arguments: frame -- the DataFrame to adjust
               adjust_dict -- a dictionary of adjustments
    Returns the adjusted DataFrame."""

    fail_if_too_long_ago = 21  # Days

    for state, (adj, date) in adjust_dict.items():
        if date > frame.index[-1]:
            frame.loc[date, state] = adj
            frame = index_missing_dates(frame)  # type: ignore[assignment]
        else:
            how_far_back = (date - frame.index[-1]) / np.timedelta64(1, "D")
            if how_far_back > fail_if_too_long_ago:
                print(f"Failed to adjust {state} by {adj} on {date}")
                continue
            frame.loc[date, state] = adj
            frame.loc[frame.index > date, state] = np.nan
            frame[state] = frame[state].ffill()

    return frame


df = make_adjustments(df, adjustments)

## Plotting

In [9]:
COUNTER = 0


def plot_rates(dataset: pd.DataFrame, start="2018-01-01") -> None:
    """Plot the central bank policy rates."""

    widths = [1.5] * 40
    if "Australia" in dataset.columns:
        widths[dataset.columns.get_loc("Australia")] = 3  # type: ignore[index]

    if len(dataset.columns) < 10:
        fs = 10
    elif len(dataset.columns) < 20:
        fs = 8
    else:
        fs = 6

    global COUNTER
    line_plot_finalise(
        dataset[lambda x: x.index >= start],
        title="Central Bank Policy Rates",
        ylabel="Annual Policy Rate (%)",
        rfooter=SOURCE,
        lfooter=LFOOTER,
        width=widths,
        style=["-", "--", ":", "-."] * 10,
        legend={"ncols": 3, "loc": "upper left", "fontsize": fs},
        y0=True,
        zero_y=True,
        tag=f"{COUNTER}",
        show=SHOW,
    )
    COUNTER += 1


comparable = [
    "Australia",
    "Canada",
    "Euro Area",
    "New Zealand",
    "United Kingdom",
    "United States",
]
bigly = [
    "United States",
    "China",
    "India",
    "Euro Area",
    "United Kingdom",
    "Japan",
    "Canada",
]
ugly = ["Argentina", "Russian Federation", "Türkiye"]
plot_rates(df[comparable])
plot_rates(df[bigly])
plot_rates(df[sorted([x for x in df.columns if x not in ugly])])
plot_rates(df[ugly])

In [10]:
MEAN_MEDIAN = 0.80  # proportion of non-na data points to plot mean and median


def plot_world(
    data: pd.DataFrame,
    exclusions: Iterable = tuple(),
    **kwargs: Any,
) -> None:
    """Plot Australia vs the BIS monitored mean and median."""

    # Exclude problematic BIS states
    keep = [x for x in data.columns if x not in exclusions]
    my_data = data[keep].copy()

    # plot remaining BIS states without legend label using the _ trick
    mapper = {x: f"_{x}" for x in my_data.columns}
    my_data = my_data.rename(columns=mapper)
    ax = my_data.plot(color="blue", lw=0.25, alpha=0.5)
    back = {y: x for x, y in mapper.items()}
    my_data = my_data.rename(columns=back)
    my_data["Australia"].dropna().plot(
        ax=ax, color="darkorange", lw=3, label="Australia"
    )

    # plot mean if THRESHOLD proportion of non-na data points met
    mean = my_data.mean(axis=1).where(
        my_data.notna().sum(axis=1) >= len(my_data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    median = my_data.median(axis=1).where(
        my_data.notna().sum(axis=1) >= len(my_data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    mean.plot(ax=ax, color="darkblue", ls="--", lw=2, label="BIS monitored mean")
    median.plot(ax=ax, color="darkred", ls=":", lw=2, label="BIS monitored median")

    # plot
    global PW_COUNTER  # yes, this is ugly
    PW_COUNTER = PW_COUNTER + 1
    finalise_plot(
        ax,
        xlabel=None,
        y0=True,
        rfooter=SOURCE,
        tag=str(PW_COUNTER),
        legend={"loc": "best", "fontsize": "xx-small"},
        **kwargs,
        show=SHOW,
    )


PW_COUNTER = 0
plot_world(
    df,
    exclusions=ugly,
    title="CB Policy Rates: Australia in World Context",
    ylabel="Annual Policy Rate (%)",
    lfooter=LFOOTER + f" Excluded: {', '.join(ugly)}.",
)

/Users/bryanpalmer/ABS/.venv/lib/python3.13/site-packages/mgplot/finalise_plot.py:374: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  fig.tight_layout(pad=TIGHT_LAYOUT_PAD)


## The End

In [11]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-09-04 08:48:13

Python implementation: CPython
Python version       : 3.13.6
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pycountry : 24.6.1
pathlib   : 1.0.1
numpy     : 2.3.2
mgplot    : 0.2.12
dateutil  : 2.9.0.post0
matplotlib: 3.10.5
pandas    : 2.3.1
typing    : 3.10.0.0

Watermark: 2.5.0

